In [20]:
import pandas as pd
from arcgis import GIS
gis = GIS()
from arcgis.features import FeatureLayer
import requests
import arcpy

In [21]:
strRelease = "WF TDM v9.0.1"
lstFieldsFromForecasts = ['SEGID','MF2023','MF2028','MF2032','MF2042','MF2050','ADJ2023','ADJ2028','ADJ2032','ADJ2042','ADJ2050','ADJHIST']

In [22]:
# read segments
sdfSegments = pd.DataFrame.spatial.from_featureclass('data/segments/WFv901_Segments_20240226_GeoKeys_AADTs_Factors.shp')
display(sdfSegments.head())

,FID,Id,SEGID,BMP,EMP,DISTANCE,CO_FIPS,ROUTE,SUBAREAID,PLANAREA,...,FAC_SUM,FAC_SUN,FAC_THU,FAC_TUE,FAC_WDAVG,FAC_WEAVG,FAC_WED,FAC_WEMAX,FAC_WIN,SHAPE
0,0,0,0006_141.0,141.035,146.868,5.843301,49.0,0006,1.0,MAG,...,1.0914,0.8819,1.012,0.9628,0.984,0.9734,0.9773,1.0649,0.8495,"{""paths"": [[[405824.11000000034, 4423860.33], ..."
1,1,0,0006_146.9,146.868,149.902,3.039023,49.0,0006,1.0,MAG,...,1.0914,0.8819,1.012,0.9628,0.984,0.9734,0.9773,1.0649,0.8495,"{""paths"": [[[413442.55030000024, 4422753.7282]..."
2,2,0,0006_149.9,149.902,150.580,0.677983,49.0,0006,1.0,MAG,...,1.0914,0.8819,1.012,0.9628,0.984,0.9734,0.9773,1.0649,0.8495,"{""paths"": [[[418330.7999999998, 4422866], [418..."
3,3,0,0006_150.6,150.580,152.555,1.973104,49.0,0006,1.0,MAG,...,1.0914,0.8819,1.012,0.9628,0.984,0.9734,0.9773,1.0649,0.8495,"{""paths"": [[[419421.8803000003, 4422872.2963],..."
4,4,0,0006_152.6,152.555,152.871,0.316275,49.0,0006,1.0,MAG,...,1.0914,0.8819,1.012,0.9628,0.984,0.9734,0.9773,1.0649,0.8495,"{""paths"": [[[422596.89969999995, 4422889.2949]..."


In [23]:
# Connect to GIS
gis = GIS()

# Access the feature layer
feature_layer_url = 'https://services1.arcgis.com/taguadKoI1XFwivx/arcgis/rest/services/ForecastsWFRCFinal_gdb/FeatureServer/0'
feature_layer = FeatureLayer(feature_layer_url)

# Convert to Spatially Enabled DataFrame
sdfForecasts = feature_layer.query().sdf
sdfForecasts = sdfForecasts[lstFieldsFromForecasts] # filter columns by list

# Display the first few rows of the dataframe
display(sdfForecasts.head())


,SEGID,MF2023,MF2028,MF2032,MF2042,MF2050,ADJ2023,ADJ2028,ADJ2032,ADJ2042,ADJ2050,ADJHIST
0,0006_141.0,1600.0,1700.0,1800.0,2100.0,2400.0,0.0,100.0,0.0,0.0,0.0,0.0
1,0006_146.9,1700.0,1700.0,1900.0,2100.0,2400.0,0.0,100.0,0.0,0.0,0.0,0.0
2,0006_149.9,2600.0,2900.0,2900.0,3500.0,6100.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0006_150.6,2600.0,2900.0,3000.0,3600.0,6200.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0006_152.6,2500.0,2900.0,2900.0,3500.0,5900.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
years = [2023, 2028, 2032, 2042, 2050]

for year in years:
    # F  : Forecast 
    # MF : Model Forecast (model volume + base year adjustment)
    # ADJ: Manual Adjustment
    sdfForecasts[f'F{year}'] = sdfForecasts[f'MF{year}'] + sdfForecasts[f'ADJ{year}'] + sdfForecasts['ADJHIST']

sdfForecasts

,SEGID,MF2023,MF2028,MF2032,MF2042,MF2050,ADJ2023,ADJ2028,ADJ2032,ADJ2042,ADJ2050,ADJHIST,F2023,F2028,F2032,F2042,F2050
0,0006_141.0,1600.0,1700.0,1800.0,2100.0,2400.0,0.0,100.0,0.0,0.0,0.0,0.0,1600.0,1800.0,1800.0,2100.0,2400.0
1,0006_146.9,1700.0,1700.0,1900.0,2100.0,2400.0,0.0,100.0,0.0,0.0,0.0,0.0,1700.0,1800.0,1900.0,2100.0,2400.0
2,0006_149.9,2600.0,2900.0,2900.0,3500.0,6100.0,0.0,0.0,0.0,0.0,0.0,0.0,2600.0,2900.0,2900.0,3500.0,6100.0
3,0006_150.6,2600.0,2900.0,3000.0,3600.0,6200.0,0.0,0.0,0.0,0.0,0.0,0.0,2600.0,2900.0,3000.0,3600.0,6200.0
4,0006_152.6,2500.0,2900.0,2900.0,3500.0,5900.0,0.0,0.0,0.0,0.0,0.0,0.0,2500.0,2900.0,2900.0,3500.0,5900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,WFRC_8469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4976,WFRC_8470,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4977,WFRC_8471,0.0,0.0,13500.0,16500.0,18500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13500.0,16500.0,18500.0
4978,WFRC_8472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
sdfForecasts.columns

Index(['SEGID', 'MF2023', 'MF2028', 'MF2032', 'MF2042', 'MF2050', 'ADJ2023',
       'ADJ2028', 'ADJ2032', 'ADJ2042', 'ADJ2050', 'ADJHIST', 'F2023', 'F2028',
       'F2032', 'F2042', 'F2050'],
      dtype='object')

In [26]:
sdfForecastsPlusSegments = pd.merge(sdfForecasts,sdfSegments,on='SEGID',how='right')
sdfForecastsPlusSegments

,SEGID,MF2023,MF2028,MF2032,MF2042,MF2050,ADJ2023,ADJ2028,ADJ2032,ADJ2042,...,FAC_SUM,FAC_SUN,FAC_THU,FAC_TUE,FAC_WDAVG,FAC_WEAVG,FAC_WED,FAC_WEMAX,FAC_WIN,SHAPE
0,0006_141.0,1600.0,1700.0,1800.0,2100.0,2400.0,0.0,100.0,0.0,0.0,...,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495,"{'paths': [[[405824.11000000034, 4423860.33], ..."
1,0006_146.9,1700.0,1700.0,1900.0,2100.0,2400.0,0.0,100.0,0.0,0.0,...,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495,"{'paths': [[[413442.55030000024, 4422753.7282]..."
2,0006_149.9,2600.0,2900.0,2900.0,3500.0,6100.0,0.0,0.0,0.0,0.0,...,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495,"{'paths': [[[418330.7999999998, 4422866], [418..."
3,0006_150.6,2600.0,2900.0,3000.0,3600.0,6200.0,0.0,0.0,0.0,0.0,...,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495,"{'paths': [[[419421.8803000003, 4422872.2963],..."
4,0006_152.6,2500.0,2900.0,2900.0,3500.0,5900.0,0.0,0.0,0.0,0.0,...,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495,"{'paths': [[[422596.89969999995, 4422889.2949]..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,WFRC_8469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0283,0.6061,1.1071,1.0813,1.0924,0.7653,1.0887,0.9245,0.9370,"{'paths': [[[407213.3742000004, 4504030.9891],..."
4976,WFRC_8470,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0575,0.6429,1.1132,1.0744,1.0918,0.7585,1.0878,0.8742,0.8994,"{'paths': [[[407723.2000000002, 4503422.753000..."
4977,WFRC_8471,0.0,0.0,13500.0,16500.0,18500.0,0.0,0.0,0.0,0.0,...,1.0283,0.6061,1.1071,1.0813,1.0924,0.7653,1.0887,0.9245,0.9370,"{'paths': [[[415734.59509999957, 4481608.6051]..."
4978,WFRC_8472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0135,0.6293,1.1063,1.0851,1.0946,0.7583,1.0925,0.8873,0.9653,"{'paths': [[[424308.5, 4489341.199999999], [42..."


In [27]:
# cacluate and adjusted base year
sdfForecastsPlusSegments['F2019'] = sdfForecastsPlusSegments['AADT2019'] + sdfForecastsPlusSegments['ADJHIST']

In [28]:
# calculate change fields
periods = [
    ('19TO50', 'F2019', 'F2050'),
    ('19TO23', 'F2019', 'F2023'),
    ('23TO28', 'F2023', 'F2028'),
    ('28TO32', 'F2028', 'F2032'),
    ('32TO42', 'F2032', 'F2042'),
    ('42TO50', 'F2042', 'F2050')
]

for period, start_year_key, end_year_key in periods:
    ch_key = f'CH{period}'
    sdfForecastsPlusSegments[ch_key] = sdfForecastsPlusSegments[end_year_key] - sdfForecastsPlusSegments[start_year_key]

sdfForecastsPlusSegments


,SEGID,MF2023,MF2028,MF2032,MF2042,MF2050,ADJ2023,ADJ2028,ADJ2032,ADJ2042,...,FAC_WEMAX,FAC_WIN,SHAPE,F2019,CH19TO50,CH19TO23,CH23TO28,CH28TO32,CH32TO42,CH42TO50
0,0006_141.0,1600.0,1700.0,1800.0,2100.0,2400.0,0.0,100.0,0.0,0.0,...,1.0649,0.8495,"{'paths': [[[405824.11000000034, 4423860.33], ...",1517.0,883.0,83.0,200.0,0.0,300.0,300.0
1,0006_146.9,1700.0,1700.0,1900.0,2100.0,2400.0,0.0,100.0,0.0,0.0,...,1.0649,0.8495,"{'paths': [[[413442.55030000024, 4422753.7282]...",1517.0,883.0,183.0,100.0,100.0,200.0,300.0
2,0006_149.9,2600.0,2900.0,2900.0,3500.0,6100.0,0.0,0.0,0.0,0.0,...,1.0649,0.8495,"{'paths': [[[418330.7999999998, 4422866], [418...",2441.0,3659.0,159.0,300.0,0.0,600.0,2600.0
3,0006_150.6,2600.0,2900.0,3000.0,3600.0,6200.0,0.0,0.0,0.0,0.0,...,1.0649,0.8495,"{'paths': [[[419421.8803000003, 4422872.2963],...",2441.0,3759.0,159.0,300.0,100.0,600.0,2600.0
4,0006_152.6,2500.0,2900.0,2900.0,3500.0,5900.0,0.0,0.0,0.0,0.0,...,1.0649,0.8495,"{'paths': [[[422596.89969999995, 4422889.2949]...",2417.0,3483.0,83.0,400.0,0.0,600.0,2400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,WFRC_8469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.9245,0.9370,"{'paths': [[[407213.3742000004, 4504030.9891],...",0.0,0.0,0.0,0.0,0.0,0.0,0.0
4976,WFRC_8470,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.8742,0.8994,"{'paths': [[[407723.2000000002, 4503422.753000...",0.0,0.0,0.0,0.0,0.0,0.0,0.0
4977,WFRC_8471,0.0,0.0,13500.0,16500.0,18500.0,0.0,0.0,0.0,0.0,...,0.9245,0.9370,"{'paths': [[[415734.59509999957, 4481608.6051]...",0.0,18500.0,0.0,0.0,13500.0,3000.0,2000.0
4978,WFRC_8472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.8873,0.9653,"{'paths': [[[424308.5, 4489341.199999999], [42...",0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
lstDropFields = [item for item in lstFieldsFromForecasts if item != 'SEGID']
sdfForecastsPlusSegments.drop(columns=lstDropFields, inplace=True)
sdfForecastsPlusSegments

,SEGID,F2023,F2028,F2032,F2042,F2050,FID,Id,BMP,EMP,...,FAC_WEMAX,FAC_WIN,SHAPE,F2019,CH19TO50,CH19TO23,CH23TO28,CH28TO32,CH32TO42,CH42TO50
0,0006_141.0,1600.0,1800.0,1800.0,2100.0,2400.0,0,0,141.035,146.868,...,1.0649,0.8495,"{'paths': [[[405824.11000000034, 4423860.33], ...",1517.0,883.0,83.0,200.0,0.0,300.0,300.0
1,0006_146.9,1700.0,1800.0,1900.0,2100.0,2400.0,1,0,146.868,149.902,...,1.0649,0.8495,"{'paths': [[[413442.55030000024, 4422753.7282]...",1517.0,883.0,183.0,100.0,100.0,200.0,300.0
2,0006_149.9,2600.0,2900.0,2900.0,3500.0,6100.0,2,0,149.902,150.580,...,1.0649,0.8495,"{'paths': [[[418330.7999999998, 4422866], [418...",2441.0,3659.0,159.0,300.0,0.0,600.0,2600.0
3,0006_150.6,2600.0,2900.0,3000.0,3600.0,6200.0,3,0,150.580,152.555,...,1.0649,0.8495,"{'paths': [[[419421.8803000003, 4422872.2963],...",2441.0,3759.0,159.0,300.0,100.0,600.0,2600.0
4,0006_152.6,2500.0,2900.0,2900.0,3500.0,5900.0,4,0,152.555,152.871,...,1.0649,0.8495,"{'paths': [[[422596.89969999995, 4422889.2949]...",2417.0,3483.0,83.0,400.0,0.0,600.0,2400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,WFRC_8469,0.0,0.0,0.0,0.0,0.0,4975,0,0.000,0.000,...,0.9245,0.9370,"{'paths': [[[407213.3742000004, 4504030.9891],...",0.0,0.0,0.0,0.0,0.0,0.0,0.0
4976,WFRC_8470,0.0,0.0,0.0,0.0,0.0,4976,0,0.000,0.000,...,0.8742,0.8994,"{'paths': [[[407723.2000000002, 4503422.753000...",0.0,0.0,0.0,0.0,0.0,0.0,0.0
4977,WFRC_8471,0.0,0.0,13500.0,16500.0,18500.0,4977,0,0.000,0.000,...,0.9245,0.9370,"{'paths': [[[415734.59509999957, 4481608.6051]...",0.0,18500.0,0.0,0.0,13500.0,3000.0,2000.0
4978,WFRC_8472,0.0,0.0,0.0,0.0,0.0,4978,0,0.000,0.000,...,0.8873,0.9653,"{'paths': [[[424308.5, 4489341.199999999], [42...",0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
sdfForecastsPlusSegments.columns


# Set display options to ensure no truncation
pd.set_option('display.max_columns', None)  # Ensures all columns are shown
pd.set_option('display.max_seq_items', None)  # For potentially long sequences

# Now, when you print or display the DataFrame, all column names will be shown
print(sdfForecastsPlusSegments.columns)

# Reset display options to default
pd.reset_option('display.max_columns')
pd.reset_option('display.max_seq_items')


Index(['SEGID', 'F2023', 'F2028', 'F2032', 'F2042', 'F2050', 'FID', 'Id',
       'BMP', 'EMP', 'DISTANCE', 'CO_FIPS', 'ROUTE', 'SUBAREAID', 'PLANAREA',
       'AADTSTN', 'AADT2022', 'AADT2021', 'AADT2020', 'AADT2019', 'AADT2018',
       'AADT2017', 'AADT2016', 'AADT2015', 'AADT2014', 'AADT2013', 'AADT2012',
       'AADT2011', 'AADT2010', 'AADT2009', 'AADT2008', 'AADT2007', 'AADT2006',
       'AADT2005', 'AADT2004', 'AADT2003', 'AADT2002', 'AADT2001', 'AADT2000',
       'AADT1999', 'AADT1998', 'AADT1997', 'AADT1996', 'AADT1995', 'AADT1994',
       'AADT1993', 'AADT1992', 'AADT1991', 'AADT1990', 'AADT1989', 'AADT1988',
       'AADT1987', 'AADT1986', 'AADT1985', 'AADT1984', 'AADT1983', 'AADT1982',
       'AADT1981', 'SUTRK2022', 'CUTRK2022', 'SUTRK2021', 'CUTRK2021',
       'SUTRK2020', 'CUTRK2020', 'SUTRK2019', 'CUTRK2019', 'SUTRK2018',
       'CUTRK2018', 'SUTRK2017', 'CUTRK2017', 'SUTRK2016', 'CUTRK2016',
       'SUTRK2015', 'CUTRK2015', 'SUTRK2014', 'CUTRK2014', 'SUTRK2013',
       'C

In [39]:
lstFinalCols = ['SEGID',
                'ROUTE', 'BMP', 'EMP', 'DISTANCE', 'CO_FIPS', 'SUBAREAID', 'PLANAREA',
                'F2019', 'F2023', 'F2028', 'F2032', 'F2042', 'F2050',
                'CH19TO50', 'CH19TO23', 'CH23TO28', 'CH28TO32', 'CH32TO42', 'CH42TO50', 
                'AADTSTN',
                'AADT2022', 'AADT2021', 'AADT2020', 'AADT2019', 'AADT2018', 'AADT2017',
                'AADT2016', 'AADT2015', 'AADT2014', 'AADT2013', 'AADT2012', 'AADT2011',
                'AADT2010', 'AADT2009', 'AADT2008', 'AADT2007', 'AADT2006', 'AADT2005',
                'AADT2004', 'AADT2003', 'AADT2002', 'AADT2001', 'AADT2000', 'AADT1999',
                'AADT1998', 'AADT1997', 'AADT1996', 'AADT1995', 'AADT1994', 'AADT1993',
                'AADT1992', 'AADT1991', 'AADT1990', 'AADT1989', 'AADT1988', 'AADT1987',
                'AADT1986', 'AADT1985', 'AADT1984', 'AADT1983', 'AADT1982', 'AADT1981',
                'SUTRK2022', 'CUTRK2022', 'SUTRK2021', 'CUTRK2021', 'SUTRK2020',
                'CUTRK2020', 'SUTRK2019', 'CUTRK2019', 'SUTRK2018', 'CUTRK2018',
                'SUTRK2017', 'CUTRK2017', 'SUTRK2016', 'CUTRK2016', 'SUTRK2015',
                'CUTRK2015', 'SUTRK2014', 'CUTRK2014', 'SUTRK2013', 'CUTRK2013',
                'SUTRK2012', 'CUTRK2012', 'SUTRK2011', 'CUTRK2011', 'SUTRK2010',
                'CUTRK2010', 'CCSGROUP19',
                'FAC_MON', 'FAC_TUE', 'FAC_WED', 'FAC_THU', 'FAC_FRI', 'FAC_SAT', 'FAC_SUN',
                'FAC_WDAVG', 'FAC_WEAVG', 'FAC_WEMAX',
                'FAC_SPR', 'FAC_SUM', 'FAC_FAL', 'FAC_WIN', 
                'FAC_JAN', 'FAC_FEB', 'FAC_MAR', 'FAC_APR', 'FAC_MAY', 'FAC_JUN',
                'FAC_JUL', 'FAC_AUG', 'FAC_SEP', 'FAC_OCT', 'FAC_NOV', 'FAC_DEC',
                'FAC_MAXMO','FAC_MAX',  
                'FACMANADJ', 'SHAPE']


sdfForecastsPlusSegmentsFinal = sdfForecastsPlusSegments[lstFinalCols].copy()
sdfForecastsPlusSegmentsFinal

,SEGID,ROUTE,BMP,EMP,DISTANCE,CO_FIPS,SUBAREAID,PLANAREA,F2019,F2023,...,FAC_JUL,FAC_AUG,FAC_SEP,FAC_OCT,FAC_NOV,FAC_DEC,FAC_MAXMO,FAC_MAX,FACMANADJ,SHAPE
0,0006_141.0,0006,141.035,146.868,5.843301,49.0,1.0,MAG,1517.0,1600.0,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{'paths': [[[405824.11000000034, 4423860.33], ..."
1,0006_146.9,0006,146.868,149.902,3.039023,49.0,1.0,MAG,1517.0,1700.0,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{'paths': [[[413442.55030000024, 4422753.7282]..."
2,0006_149.9,0006,149.902,150.580,0.677983,49.0,1.0,MAG,2441.0,2600.0,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{'paths': [[[418330.7999999998, 4422866], [418..."
3,0006_150.6,0006,150.580,152.555,1.973104,49.0,1.0,MAG,2441.0,2600.0,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{'paths': [[[419421.8803000003, 4422872.2963],..."
4,0006_152.6,0006,152.555,152.871,0.316275,49.0,1.0,MAG,2417.0,2500.0,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{'paths': [[[422596.89969999995, 4422889.2949]..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,WFRC_8469,WFRC,0.000,0.000,0.567664,35.0,1.0,WFRC,0.0,0.0,...,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9,1.0434,0.0,"{'paths': [[[407213.3742000004, 4504030.9891],..."
4976,WFRC_8470,WFRC,0.000,0.000,0.493148,35.0,1.0,WFRC,0.0,0.0,...,1.0366,1.0724,1.0607,1.0468,0.9896,0.9399,8,1.0724,0.0,"{'paths': [[[407723.2000000002, 4503422.753000..."
4977,WFRC_8471,WFRC,0.000,0.000,0.405891,35.0,1.0,WFRC,0.0,0.0,...,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9,1.0434,0.0,"{'paths': [[[415734.59509999957, 4481608.6051]..."
4978,WFRC_8472,WFRC,0.000,0.000,0.253734,35.0,1.0,WFRC,0.0,0.0,...,0.9960,1.0223,1.0238,1.0103,0.9944,0.9907,5,1.0245,0.0,"{'paths': [[[424308.5, 4489341.199999999], [42..."


In [40]:
sdfForecastsPlusSegmentsFinal.spatial.to_featureclass('results/Traffic-Volume-Historic-and-Forecast.shp',sanitize_columns=False)

'e:\\GitHub\\Traffic-Volume-Forecasts\\results\\Traffic-Volume-Historic-and-Forecast.shp'